# Collecting Textual News Data from Twitter


## Importing Dependencies

In [1]:
from dotenv import load_dotenv
# import requests
# import json
import os
import tweepy
# import config
import pandas as pd

pd.options.display.max_columns = 999

## Setting Up Twitter API and Tokens from .env File

In [2]:
load_dotenv()

True

## Setting Up Twitter Screaper

In [349]:
def getClient():
    
    client = tweepy.Client(bearer_token= os.getenv("BEARER_TOKEN"),
                           consumer_key=os.getenv("API_KEY"), 
                           consumer_secret=os.getenv("API_KEY_SECRET"), 
                           access_token=os.getenv("ACCESS_TOKEN"),
                           access_token_secret=os.getenv("ACCESS_TOKEN_SECRET"))

    return client


def searchTweets(query, qty):
    
    client = getClient()
    
    client_call = client.search_all_tweets(query=query, max_results=qty, expansions='entities.mentions.username')
    
    
    
    
    
    
    # tweets_by_users = client.get_users_tweets(id=612473, max_results=5)
                                        
    tweet_data = client_call.data
    user_data = client_call

    tweets = []
    ids = []
    tokens = []
    users = []
    
    

    if not tweet_data is None and len(tweet_data) > 0:
        for tweet in tweet_data:
            
            tweets.append(tweet.text)
            ids.append(tweet.id)
            
        for u in range(0, len(client_call.includes['users'])):
                users.append(client_call.includes['users'][u].username)
            
        
    else:
        return []
    
    df = pd.DataFrame({'Tweets': tweets, 'ID': ids, 'Usernames': users})
    df = df[~df.Tweets.str.contains("RT")]
    
    
    
    
    
    
    
    
                          
    return usersssss


# Calling Function to search for tweets
This function will get all tweets in a dataframe and then it will be exported on a CSV file

In [350]:
df = searchTweets('covid-19 news lang=en -is:retweet', 10)
    

ValueError: All arrays must be of the same length

In [230]:
df.to_csv('Data_1/BBC_BreakinNews_Collected_Data22222.csv')

AttributeError: 'list' object has no attribute 'to_csv'

In [347]:
df

['Labourheartland',
 'davmidd',
 'redsarah99',
 'Gaynorann46',
 'Hardeep216',
 'Hawkins2Mac',
 'PSIglobalunion']